<a href="https://colab.research.google.com/github/maengg/Kaggle_Kernel_Study/blob/main/Porto_Data_Preparation_%26_Exploration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

***운전자가 내년에 보험금을 청구할 것인지 예측.***

평가 지표 : Normalized Gini Coefficient

***gini = 2 X AUC-1*** 이다. 즉, 모델이 랜덤하게 찍는 경우 AUC는 0.5지만, 지니 계수를 사용하면 0이 나온다. (보기 더 좋으니까 AUC 대신 사용하는 거 아니겠냐는 말이 있다.)

ROC AUC Curve는 threshold를 0~1까지 다 따지는 것. threshold 값 별로 재현율과 위양성률이 어떻게 되는 지 점으로 찍은 후 쭉 그리면 Curve가 나온다. 이때 이 Curve 밑 면적이 score가 된다.

즉, 지니 계수는 ROC AUC Score와 거의 비슷한 목적으로 쓰인다. (랜덤으로 찍는 경우 - 0, 퍼펙트 - 0.5)

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# null 채우는 것.
from sklearn.impute import SimpleImputer
# 중요한 numerical 특성들을 곱하는 것.
from sklearn.preprocessing import PolynomialFeatures 
from sklearn.preprocessing import StandardScaler
# 중요한 특성들만 뽑기 위한 것.
from sklearn.feature_selection import VarianceThreshold # 특성의 분산이 작다 -> 정보가 작다. 스펙트럼이 넓어야 얻을 수 있는 정보가 많다. 0과 1인데 99%가 1이면 별로 얻을 정보가 없다.(분산이 작다.)
# 특성의 중요도에 따라 Selection 한다.
from sklearn.feature_selection import SelectFromModel
from sklearn.utils import shuffle
from sklearn.ensemble import RandomForestClassifier

pd.set_option('display.max_columns', 100)

In [10]:
train = pd.read_csv('/content/drive/MyDrive/train.csv')
test = pd.read_csv('/content/drive/MyDrive/test.csv')

In [11]:
train.head()

,id,target,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,ps_ind_10_bin,ps_ind_11_bin,ps_ind_12_bin,ps_ind_13_bin,ps_ind_14,ps_ind_15,ps_ind_16_bin,ps_ind_17_bin,ps_ind_18_bin,ps_reg_01,ps_reg_02,ps_reg_03,ps_car_01_cat,ps_car_02_cat,ps_car_03_cat,ps_car_04_cat,ps_car_05_cat,ps_car_06_cat,ps_car_07_cat,ps_car_08_cat,ps_car_09_cat,ps_car_10_cat,ps_car_11_cat,ps_car_11,ps_car_12,ps_car_13,ps_car_14,ps_car_15,ps_calc_01,ps_calc_02,ps_calc_03,ps_calc_04,ps_calc_05,ps_calc_06,ps_calc_07,ps_calc_08,ps_calc_09,ps_calc_10,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
0,7,0,2,2,5,1,0,0,1,0,0,0,0,0,0,0,11,0,1,0,0.7,0.2,0.718070,10,1,-1,0,1,4,1,0,0,1,12,2,0.400000,0.883679,0.370810,3.605551,0.6,0.5,0.2,3,1,10,1,10,1,5,9,1,5,8,0,1,1,0,0,1
1,9,0,1,1,7,0,0,0,0,1,0,0,0,0,0,0,3,0,0,1,0.8,0.4,0.766078,11,1,-1,0,-1,11,1,1,2,1,19,3,0.316228,0.618817,0.388716,2.449490,0.3,0.1,0.3,2,1,9,5,8,1,7,3,1,1,9,0,1,1,0,1,0
2,13,0,5,4,9,1,0,0,0,1,0,0,0,0,0,0,12,1,0,0,0.0,0.0,-1.000000,7,1,-1,0,-1,14,1,1,2,1,60,1,0.316228,0.641586,0.347275,3.316625,0.5,0.7,0.1,2,2,9,1,8,2,7,4,2,7,7,0,1,1,0,1,0
3,16,0,0,1,2,0,0,1,0,0,0,0,0,0,0,0,8,1,0,0,0.9,0.2,0.580948,7,1,0,0,1,11,1,1,3,1,104,1,0.374166,0.542949,0.294958,2.000000,0.6,0.9,0.1,2,4,7,1,8,4,2,2,2,4,9,0,0,0,0,0,0
4,17,0,0,2,0,1,0,1,0,0,0,0,0,0,0,0,9,1,0,0,0.7,0.6,0.840759,11,1,-1,0,-1,14,1,1,2,1,82,3,0.316070,0.565832,0.365103,2.000000,0.4,0.6,0.0,2,2,6,3,10,2,12,3,1,1,3,0,0,0,1,1,0


In [12]:
cat_cols = [col for col in train.columns if 'cat' in col]

In [13]:
# 카테고리컬 특성에서 카디날리티 확인.
for col in cat_cols:
  print(col, train[col].nunique())

ps_ind_02_cat 5
ps_ind_04_cat 3
ps_ind_05_cat 8
ps_car_01_cat 13
ps_car_02_cat 3
ps_car_03_cat 3
ps_car_04_cat 10
ps_car_05_cat 3
ps_car_06_cat 18
ps_car_07_cat 3
ps_car_08_cat 2
ps_car_09_cat 6
ps_car_10_cat 3
ps_car_11_cat 104


In [14]:
train.shape, test.shape

((595212, 59), (892816, 58))

In [15]:
# 중복 없음.
train.drop_duplicates()
train.shape

(595212, 59)

In [16]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 595212 entries, 0 to 595211
Data columns (total 59 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   id              595212 non-null  int64  
 1   target          595212 non-null  int64  
 2   ps_ind_01       595212 non-null  int64  
 3   ps_ind_02_cat   595212 non-null  int64  
 4   ps_ind_03       595212 non-null  int64  
 5   ps_ind_04_cat   595212 non-null  int64  
 6   ps_ind_05_cat   595212 non-null  int64  
 7   ps_ind_06_bin   595212 non-null  int64  
 8   ps_ind_07_bin   595212 non-null  int64  
 9   ps_ind_08_bin   595212 non-null  int64  
 10  ps_ind_09_bin   595212 non-null  int64  
 11  ps_ind_10_bin   595212 non-null  int64  
 12  ps_ind_11_bin   595212 non-null  int64  
 13  ps_ind_12_bin   595212 non-null  int64  
 14  ps_ind_13_bin   595212 non-null  int64  
 15  ps_ind_14       595212 non-null  int64  
 16  ps_ind_15       595212 non-null  int64  
 17  ps_ind_16_

## Metadata

데이터 관리를 용이하게 하기 위해 variables의 정보들을 데이터프레임 형태로 저장.

In [18]:
data = []

for f in train.columns:
  if f == 'target':
    role = 'target'
  elif f == 'id':
    role = 'id'
  else:
    role = 'input'

  # 레벨 정의
  if 'bin' in f or f == 'target':
    level = 'binary'
  elif 'cat' in f or f =='id':
    level = 'norminal'
  elif train[f].dtype == float:
    level = 'interval'
  elif train[f].dtype == int:
    level = 'ordinal'
  # keep => 해당 feature를 쓸지 안 쓸지.
  keep = True
  if f =='id':
    keep = False

  dtype = train[f].dtype

  f_dict = {
      'varname' : f,
      'role' : role,
      'level' : level,
      'keep' : keep,
      'dtype' : dtype
  }

  data.append(f_dict)

In [20]:
meta = pd.DataFrame(data, columns =['varname', 'role', 'level', 'keep', 'dtype'])
meta.set_index('varname', inplace=True)
meta

,role,level,keep,dtype
varname,,,,
id,id,norminal,False,int64
target,target,binary,True,int64
ps_ind_01,input,ordinal,True,int64
ps_ind_02_cat,input,norminal,True,int64
ps_ind_03,input,ordinal,True,int64
ps_ind_04_cat,input,norminal,True,int64
ps_ind_05_cat,input,norminal,True,int64
ps_ind_06_bin,input,binary,True,int64
ps_ind_07_bin,input,binary,True,int64


In [24]:
# Ex) level이 norminal인 데이터의 인덱스 추출
meta.loc[(meta.level=='norminal') & (meta.keep)].index

Index(['ps_ind_02_cat', 'ps_ind_04_cat', 'ps_ind_05_cat', 'ps_car_01_cat',
       'ps_car_02_cat', 'ps_car_03_cat', 'ps_car_04_cat', 'ps_car_05_cat',
       'ps_car_06_cat', 'ps_car_07_cat', 'ps_car_08_cat', 'ps_car_09_cat',
       'ps_car_10_cat', 'ps_car_11_cat'],
      dtype='object', name='varname')

In [25]:
# role과 level에 따른 target의 수
pd.DataFrame({'count' : meta.groupby(['role', 'level'])['role'].size()}).reset_index()

,role,level,count
0,id,norminal,1
1,input,binary,17
2,input,interval,10
3,input,norminal,14
4,input,ordinal,16
5,target,binary,1


In [26]:
# metadata 사용하여 기술통계량 계산.
v = meta[(meta.level=='interval') & (meta.keep)].index
train[v].describe()

,ps_reg_01,ps_reg_02,ps_reg_03,ps_car_12,ps_car_13,ps_car_14,ps_car_15,ps_calc_01,ps_calc_02,ps_calc_03
count,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000
mean,0.610991,0.439184,0.551102,0.379945,0.813265,0.276256,3.065899,0.449756,0.449589,0.449849
std,0.287643,0.404264,0.793506,0.058327,0.224588,0.357154,0.731366,0.287198,0.286893,0.287153
min,0.000000,0.000000,-1.000000,-1.000000,0.250619,-1.000000,0.000000,0.000000,0.000000,0.000000
25%,0.400000,0.200000,0.525000,0.316228,0.670867,0.333167,2.828427,0.200000,0.200000,0.200000
50%,0.700000,0.300000,0.720677,0.374166,0.765811,0.368782,3.316625,0.500000,0.400000,0.500000
75%,0.900000,0.600000,1.000000,0.400000,0.906190,0.396485,3.605551,0.700000,0.700000,0.700000
max,0.900000,1.800000,4.037945,1.264911,3.720626,0.636396,3.741657,0.900000,0.900000,0.900000


reg variables
- ps_reg_03 결측치를 가지고 있음.
- 스케일링 필요해보인다.

car variables
- ps_car_12, ps_car_14 결측치 가지고 있음.
- 스케일링 필요해보인다.

calc variables
- 결측치 없음.
- 최대값이 0.9인 것으로 봐서 비율인 것 같다.
- 세 변수들이 비슷한 분포를 가지고 있다.

In [27]:
v = meta[(meta.level=='ordinal') & (meta.keep)].index
train[v].describe()

,ps_ind_01,ps_ind_03,ps_ind_14,ps_ind_15,ps_car_11,ps_calc_04,ps_calc_05,ps_calc_06,ps_calc_07,ps_calc_08,ps_calc_09,ps_calc_10,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14
count,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000
mean,1.900378,4.423318,0.012451,7.299922,2.346072,2.372081,1.885886,7.689445,3.005823,9.225904,2.339034,8.433590,5.441382,1.441918,2.872288,7.539026
std,1.983789,2.699902,0.127545,3.546042,0.832548,1.117219,1.134927,1.334312,1.414564,1.459672,1.246949,2.904597,2.332871,1.202963,1.694887,2.746652
min,0.000000,0.000000,0.000000,0.000000,-1.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,2.000000,0.000000,5.000000,2.000000,2.000000,1.000000,7.000000,2.000000,8.000000,1.000000,6.000000,4.000000,1.000000,2.000000,6.000000
50%,1.000000,4.000000,0.000000,7.000000,3.000000,2.000000,2.000000,8.000000,3.000000,9.000000,2.000000,8.000000,5.000000,1.000000,3.000000,7.000000
75%,3.000000,6.000000,0.000000,10.000000,3.000000,3.000000,3.000000,9.000000,4.000000,10.000000,3.000000,10.000000,7.000000,2.000000,4.000000,9.000000
max,7.000000,11.000000,4.000000,13.000000,3.000000,5.000000,6.000000,10.000000,9.000000,12.000000,7.000000,25.000000,19.000000,10.000000,13.000000,23.000000


In [28]:
v = meta[(meta.level=='binary') & (meta.keep)].index
train[v].describe()

,target,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,ps_ind_10_bin,ps_ind_11_bin,ps_ind_12_bin,ps_ind_13_bin,ps_ind_16_bin,ps_ind_17_bin,ps_ind_18_bin,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
count,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000
mean,0.036448,0.393742,0.257033,0.163921,0.185304,0.000373,0.001692,0.009439,0.000948,0.660823,0.121081,0.153446,0.122427,0.627840,0.554182,0.287182,0.349024,0.153318
std,0.187401,0.488579,0.436998,0.370205,0.388544,0.019309,0.041097,0.096693,0.030768,0.473430,0.326222,0.360417,0.327779,0.483381,0.497056,0.452447,0.476662,0.360295
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000
75%,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


***target 데이터 평균이 0.0364 즉, 데이터가 불균형하다. (0이 많다.)***

## **Handling imbalanced classes**

taget = 1 인 데이터에 oversampling 하거나, target = 0 인 데이터에 undersampling 하는 방법이 있다.

데이터셋이 크기 때문에 undersampling 진행.

SMOTE

똑같은 데이터를 생성하면 모델 학습에 유의미한 데이터가 생성됐다고 보기 힘들다.
그래서 유사한 데이터를 추가하는 방식으로 Oversampling을 한다.

1. oversampling 하고자 하는 데이터 하나를 고르고 k nearest neighbor로 다섯개 찾는다.
2. 다섯개 중 아무거나 하나 고른다.
3. 처음에 고른 애와 두번째에 고른 데이터 사이의 중간지점에 데이터 하나를 생성.
4. 반복해서 진행.

In [32]:
desired_apriori=0.10

# target value의 인덱스를 추출.
idx_0 = train[train.target == 0].index
idx_1 = train[train.target == 1].index

# target value 기존 record 수를 구한다.
nb_0 = len(train.loc[idx_0]) # 573518개
nb_1 = len(train.loc[idx_1]) # 21694개

# undersampling 비율을 계산하고 target = 0 인 record 수를 계산
undersampling_rate = (((1-desired_apriori)*nb_1) / (nb_0*desired_apriori))
undersampled_nb_0 = int(undersampling_rate*nb_0)

print('Rate to undersample records with target=0 : {}'.format(undersampling_rate))
print('Number of records with target=0 after undersampling: {}'.format(undersampled_nb_0))

# shuffle을 활용하여 undersampling 된 개수만큼의 samples를 가지는 nb=0을 무작위로 추출.
undersampled_idx = shuffle(idx_0, random_state=37, n_samples=undersampled_nb_0)

# 추출한 인덱스와 기존의 idx_1을 활용하여 리스트를 만든다
idx_list = list(undersampled_idx) + list(idx_1)

# undersample 된 데이터 프레임을 돌려받는다.
train = train.loc[idx_list].reset_index(drop=True)

Rate to undersample records with target=0 : 0.34043569687437886
Number of records with target=0 after undersampling: 195246


## Data Quality Checks

**Checking missing values**

결측값 -1로 나타내지고 있음.

In [33]:
vars_with_missing = []

for f in train.columns:
  missings = train[train[f] == -1][f].count()
  if missings > 0:
    vars_with_missing.append(f)
    missings_perc = missings/train.shape[0]

    print('Variable {} has {} records ({:.2%}) with missing values'.format(f, missings, missings_perc))

print('In total, there are {} variables with missing values'.format(len(vars_with_missing)))

Variable ps_ind_02_cat has 103 records (0.05%) with missing values
Variable ps_ind_04_cat has 51 records (0.02%) with missing values
Variable ps_ind_05_cat has 2256 records (1.04%) with missing values
Variable ps_reg_03 has 38580 records (17.78%) with missing values
Variable ps_car_01_cat has 62 records (0.03%) with missing values
Variable ps_car_02_cat has 2 records (0.00%) with missing values
Variable ps_car_03_cat has 148367 records (68.39%) with missing values
Variable ps_car_05_cat has 96026 records (44.26%) with missing values
Variable ps_car_07_cat has 4431 records (2.04%) with missing values
Variable ps_car_09_cat has 230 records (0.11%) with missing values
Variable ps_car_11 has 1 records (0.00%) with missing values
Variable ps_car_14 has 15726 records (7.25%) with missing values
In total, there are 12 variables with missing values


In [34]:
# 결측치의 타겟값이 큰 의미가 없다.
train[['ps_car_05_cat', 'target']].groupby('ps_car_05_cat').mean()

,target
ps_car_05_cat,
-1,0.088018
0,0.108925
1,0.110050


- ps_car_03_cat 과 ps_car_05_cat 은 높은 결측치 비율을 가지고 있다. 따라서 삭제.
- 결측값이 있는 다른 Cat Variables는 결측값을 -1 그대로 둘 수 있다.
- ps_reg_03 은 18%의 결측값을 지니고 있다. 평균값으로 대치. (연속형 데이터)
- ps_car_11 은 5개의 결측값을 지니고 있다. ordinal의 형태이므로 평균값으로 대치하면 안됨. 최빈값으로 대치.
- ps_car_12 은 1개의 결측값을 지니고 있다. 평균값으로 대치. (연속형 데이터)
- ps_car_14 은 7%의 결측값을 지니고 있다. 평균값으로 대치. (연속형 데이터)


In [ ]:
# 너무 많은 결측값을 지닌 Feature들을 제거합

vars_to_drop = ['ps_car_03_cat', 'ps_car_05_cat']

train.drop(vars_to_drop, axis=1, inplace=True)

In [38]:
# 메타 데이터 업데이트.
meta.loc[(vars_to_drop), 'keep']

varname
ps_car_03_cat    False
ps_car_05_cat    False
Name: keep, dtype: bool

In [41]:
# 결측값 simpleimputer 활용하여 변환.
mean_imp = SimpleImputer(missing_values=-1, strategy='mean')
mode_imp = SimpleImputer(missing_values=-1, strategy='most_frequent')
# ravel() => 2차원 벡터를 풀어주는 것.
train['ps_reg_03'] = mean_imp.fit_transform(train[['ps_reg_03']]).ravel()
train['ps_car_12'] = mean_imp.fit_transform(train[['ps_car_12']]).ravel()
train['ps_car_14'] = mean_imp.fit_transform(train[['ps_car_14']]).ravel()
train['ps_car_11'] = mode_imp.fit_transform(train[['ps_car_11']]).ravel()

**Checking the cardinality of the categorical variables**

카디날리티는 전체 행에 대한 특정 컬럼의 중복 수치를 나타내는 지표.
중복도가 높으면 카디날리티가 낮고, 중복도가 낮으면 카디날리티가 높다.

따라서 카디날리티는 variable 내에서 다른 value의 개수를 말한다. 추후 categorical variables를 더미화시킬 것인데, variables 내에 다른 value들이 얼마나 많은지 체크해봐야 한다.Value들이 많을 경우, 수 많은 더미 변수들이 만들어질 수 있기 때문.

In [42]:
v = meta[(meta.level == 'norminal') & (meta.keep)].index

for f in v:
  dist_values = train[f].value_counts().shape[0]
  print('Variable {} has {} distinct values'.format(f, dist_values))

Variable ps_ind_02_cat has 5 distinct values
Variable ps_ind_04_cat has 3 distinct values
Variable ps_ind_05_cat has 8 distinct values
Variable ps_car_01_cat has 13 distinct values
Variable ps_car_02_cat has 3 distinct values
Variable ps_car_04_cat has 10 distinct values
Variable ps_car_06_cat has 18 distinct values
Variable ps_car_07_cat has 3 distinct values
Variable ps_car_08_cat has 2 distinct values
Variable ps_car_09_cat has 6 distinct values
Variable ps_car_10_cat has 3 distinct values
Variable ps_car_11_cat has 104 distinct values


**ps_car_11_cat** 104개로 매우 많은 value를 가지고 있다.

**Mean Encoding**
- 타겟값을 알고 있을때 각 카테고리들이 가지고 있는 타겟값에 대한 평균값을 취해주는 것.
- group 시킬 categorical feature와 평균을 낼 타겟값인 numerical feature가 필요하다.
- **overfitting이** 일어날 가능성이 높다.

**Frequency encoding**
- 해당 카테고리가 등장한 횟수 / 총 데이터 갯수.